In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from scipy import stats
import pickle

# Load your dataset
df = pd.read_csv("_crop+yield+prediction_data_crop_yield.csv")
print(df['Crop'].unique())

# Preprocessing steps
df['Crop'] = df['Crop'].astype('category').cat.codes
df.dropna(inplace=True)
print(df['Crop'].unique())

# Apply Z-score outlier filtering
X = df[['Crop', 'Precipitation (mm day-1)', 'Specific Humidity at 2 Meters (g/kg)', 
        'Relative Humidity at 2 Meters (%)', 'Temperature at 2 Meters (C)']]
y = df['Yield']

z_scores = np.abs(stats.zscore(X))
filtered_entries = (z_scores < 3).all(axis=1)
X = X[filtered_entries]
y = y[filtered_entries]

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=100)


['Cocoa, beans' 'Oil palm fruit' 'Rice, paddy' 'Soybean']
[0 1 2 3]


In [4]:
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

# Load your dataset
df = pd.read_csv("_crop+yield+prediction_data_crop_yield.csv")

# Preprocessing steps
df['Crop'] = df['Crop'].astype('category').cat.codes
df.dropna(inplace=True)

# Apply Z-score outlier filtering
X = df[['Crop', 'Precipitation (mm day-1)', 'Specific Humidity at 2 Meters (g/kg)', 
        'Relative Humidity at 2 Meters (%)', 'Temperature at 2 Meters (C)']]
y = df['Yield']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=100)

# Define RandomForestRegressor model and set up GridSearchCV
rf = RandomForestRegressor(random_state=100)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

best_rf = grid_search.best_estimator_

# Save the model and the scaler
with open('crop_yield_rf_model.pkl', 'wb') as model_file:
    pickle.dump(best_rf, model_file)

with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)


Fitting 5 folds for each of 324 candidates, totalling 1620 fits
